In [ ]:
import cv2
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import os
from deepface import DeepFace as df
#from retinaface import RetinaFace as m
#from PIL import Image
#import imagehash
import pandas as pd
import numpy as np
import mediapipe as mp

In [ ]:
%%time
#image denoising using opencv medianfilter

import numpy as np

def deblur_image(image):
    # Perform deblurring using your preferred method
    # This could be a function like the one provided earlier
    # For simplicity, I'll assume a placeholder deblurring function here
    deblurred_image = cv2.medianBlur(image, 5)  # Placeholder deblurring function
    return deblurred_image
 
deblurred_face = deblur_image('blurred_img_path')
plt.imshow(deblurred_face)
   

In [ ]:
#wiener filter deblurring
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage import color, data
from scipy.ndimage import gaussian_filter
from skimage import img_as_float

def wiener_filter_deblur(blurred_image, psf, regularization):
    # Convert the image to grayscale if it's in color
    if blurred_image.ndim == 3:
        blurred_image = color.rgb2gray(blurred_image)

    # Convert the PSF and the blurred image to float
    psf = psf.astype(float)
    blurred_image = blurred_image.astype(float)

    # Resize the PSF to match the size of the blurred image
    psf_resized = np.zeros_like(blurred_image)
    psf_resized[:psf.shape[0], :psf.shape[1]] = psf

    # Apply the Wiener filter to deblur the image
    wiener_filter = np.conj(psf_resized) / (np.abs(psf_resized) ** 2 + regularization)
    deblurred_image = np.real(np.fft.ifft2(np.fft.fft2(blurred_image) * np.fft.fft2(wiener_filter)))

    return deblurred_image

# Load your own blurred image
blurred_image = plt.imread('img_path')


# Define the PSF (Point Spread Function)
psf = np.ones((5, 5)) / 25  # Example PSF, you should replace it with your actual PSF

# Set the regularization parameter for the Wiener filter
regularization_param = 0.1  # Adjust this parameter based on your image characteristics

# Deblur the image using the Wiener filter
deblurred_image = wiener_filter_deblur(blurred_image, psf, regularization_param)

# Normalize the deblurred image to range [0, 1]
deblurred_image_normalized = (deblurred_image - np.min(deblurred_image)) / (np.max(deblurred_image) - np.min(deblurred_image))

deblurred_image_uint8 = (deblurred_image_normalized * 255).astype(np.uint8)
# Convert the deblurred image to RGB
deblurred_image_rgb = color.gray2rgb(deblurred_image_uint8)

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(131)
plt.imshow(blurred_image, cmap='gray')
plt.title('Blurred Image')
plt.axis('off')

plt.subplot(132)
plt.imshow(deblurred_image_rgb)
plt.title('Deblurred Image (Wiener Filter)')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#contrast enhancement using clahe
import cv2
import numpy as np

img = cv2.imread('img_path',1)
# converting to LAB color space
lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l_channel, a, b = cv2.split(lab)

# Applying CLAHE to L-channel
# feel free to try different values for the limit and grid size:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl = clahe.apply(l_channel)

# merge the CLAHE enhanced L-channel with the a and b channel
limg = cv2.merge((cl,a,b))

# Converting image from LAB Color model to BGR color spcae
enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

# Stacking the original image with the enhanced image
result = np.hstack((img, enhanced_img))
cv2.imshow('Result', result)

In [ ]:
plt.imshow(enhanced_img)